In [ ]:
import numpy as np
import argparse
import os
import imp
import re
import pickle
import datetime
import random
import math
import copy

from sklearn.model_selection import KFold, StratifiedKFold
import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset, Subset

from utils import metrics


In [ ]:
input_dim = 25
pad_token = np.zeros(input_dim)
def pad_sents(sents, pad_token):

    sents_padded = []

    max_length = max([len(_) for _ in sents])
    for i in sents:
        padded = list(i) + [pad_token]*(max_length-len(i))
        sents_padded.append(np.array(padded))

    return np.array(sents_padded)

In [ ]:
def get_bce_loss(y_pred, y_true):
    loss = torch.nn.BCELoss()
    return loss(y_pred[:,-1], y_true)

In [ ]:
def predict_last_visit_bce_loss(y_pred, x_lab_length, y_true):
    batch_size = len(y_true)
    loss = torch.nn.BCELoss()
    indices = torch.arange(batch_size, dtype=torch.int64)
    losses = 0
    for i in indices:
        losses += (loss(y_pred[i][x_lab_length[i].long()-1], y_true[i]))

    return losses/batch_size

In [ ]:
def predict_all_visits_mse_loss(y_pred, x_lab_length, y_true):
    # print('!!!')
    mini_batch_size = y_pred.shape[0]
    loss = torch.nn.MSELoss()
    indices = torch.arange(mini_batch_size, dtype=torch.int64)
    losses = 0
    # print('bs:', mini_batch_size)
    for i in indices:
        visits_indices = torch.arange(x_lab_length[i], dtype=torch.int64)
        patient_losses = 0
        for v in visits_indices:
            patient_losses += (loss(y_pred[i][v], y_true[i][v]))
        losses += patient_losses/x_lab_length[i]
    return losses/mini_batch_size

In [ ]:
class MultiTaskLoss(nn.Module):
    def __init__(self, task_num):
        super(MultiTaskLoss, self).__init__()
        self.task_num = task_num
        self.log_vars = nn.Parameter(torch.zeros((task_num)))

    def forward(self, outcome_pred, los_pred, outcome, los):

        mse, bce = torch.nn.MSELoss(), torch.nn.BCELoss()

        loss0 = bce(outcome_pred, outcome)
        loss1 = mse(los_pred, los)
    
        precision0 = torch.exp(-self.log_vars[0])
        loss0 = precision0*loss0 + self.log_vars[0]

        precision1 = torch.exp(-self.log_vars[1])
        loss1 = precision1*loss1 + self.log_vars[1]
        
        return loss0+loss1

sig = nn.Sigmoid()

x = torch.randn(3)
x = sig(x)
x2 = torch.randn(3)
y1 = torch.randn(3)
y1 = sig(y1)
y2 = torch.randn(3)

loss_fn = MultiTaskLoss(2)

optimizer = torch.optim.SGD(loss_fn.parameters(), lr = 0.1)
optimizer.zero_grad()
loss_fn(x, x2, y1, y2).backward()
optimizer.step()

In [ ]:
def predict_all_visits_outcome_bce_and_los_mse_loss(y_outcome_pred, y_los_pred, x_lab_length, y_outcome_true, y_los_true):
    mini_batch_size = y_outcome_pred.shape[0]
    loss = MultiTaskLoss(2)
    indices = torch.arange(mini_batch_size, dtype=torch.int64)
    losses = 0
    for i in indices:
        visits_indices = torch.arange(x_lab_length[i], dtype=torch.int64)
        patient_losses = 0
        for v in visits_indices:
            patient_losses += (loss(y_outcome_pred[i][v], y_los_pred[i][v], y_outcome_true[i], y_los_true[i][v]))
        losses += patient_losses/x_lab_length[i]
    return losses/mini_batch_size

In [ ]:
class Dataset(data.Dataset):
    def __init__(self, x_lab, x_lab_length, x_demo, y_outcome, y_los):
        self.x_lab = x_lab
        self.x_lab_length = x_lab_length
        self.x_demo = x_demo
        self.y_outcome = y_outcome
        self.y_los = y_los

    def __getitem__(self, index): # 返回的是tensor
        return self.x_lab[index], self.x_lab_length[index], self.x_demo[index], self.y_outcome[index], self.y_los[index]

    def __len__(self):
        return len(self.y_outcome)

In [ ]:
data_path = './dataset/tongji/processed_data/'
file_name = './ckpt/gru.pth'

batch_size = 64
num_epochs = 70
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
#device = torch.device('cpu')
print("available device: {}".format(device))


x_lab = pickle.load(open('./dataset/tongji/processed_data/train_x_labtest.pkl', 'rb'))
x_lab = np.array(x_lab, dtype=object)
x_lab = [torch.Tensor(_) for _ in x_lab]

x_demo = pickle.load(open('./dataset/tongji/processed_data/train_x_demographic.pkl', 'rb'))
x_demo = np.array(x_demo)

y_outcome = pickle.load(open('./dataset/tongji/processed_data/train_y_outcome.pkl', 'rb'))
y_outcome = np.array(y_outcome)

y_los = pickle.load(open('./dataset/tongji/processed_data/train_y_LOS.pkl', 'rb'))
y_los = np.array(y_los, dtype=object)
y_los = [torch.Tensor(_) for _ in y_los]

x_lab_length = [len(_) for _ in x_lab]
x_lab_length = np.array(x_lab_length)
x_lab = torch.nn.utils.rnn.pad_sequence((x_lab), batch_first=True)
y_los = torch.nn.utils.rnn.pad_sequence(y_los, batch_first=True)


In [ ]:
train_dataset = Dataset(x_lab, x_lab_length, x_demo, y_outcome, y_los)


In [ ]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED) # numpy
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED) # cpu
torch.cuda.manual_seed(RANDOM_SEED) # gpu
torch.backends.cudnn.deterministic=True # cudnn
np.set_printoptions(threshold=np.inf, precision=2, suppress=True)


def train_epoch(model, device, dataloader, loss_fn, optimizer):
    train_loss = []
    model.train()
    for step, data in enumerate(dataloader):   
        batch_x_lab, batch_x_lab_length, batch_x_demo, batch_y_outcome, batch_y_los = data
        batch_x_lab, batch_x_lab_length, batch_x_demo, batch_y_outcome, batch_y_los = batch_x_lab.float(),batch_x_lab_length.float() , batch_x_demo.float().to(device), batch_y_outcome.float(), batch_y_los.float()
        batch_y_outcome = batch_y_outcome.unsqueeze(-1)
        batch_y_los = batch_y_los.unsqueeze(-1)
        optimizer.zero_grad()
        outcome, los = model(batch_x_lab, batch_x_lab_length, batch_x_demo)
        loss = loss_fn(outcome, los, batch_x_lab_length, batch_y_outcome, batch_y_los)
        train_loss.append(loss.item())
        loss.backward()
        optimizer.step()
    return np.array(train_loss).mean()

def val_epoch(model, device, dataloader, loss_fn):
    val_loss = []
    y_outcome_pred = []
    y_outcome_true = []
    y_los_pred = []
    y_los_true = []
    evaluation_scores = {}
    model.eval()
    with torch.no_grad():
        for step, data in enumerate(dataloader):   
            batch_x_lab, batch_x_lab_length, batch_x_demo, batch_y_outcome, batch_y_los = data
            batch_x_lab, batch_x_lab_length, batch_x_demo, batch_y_outcome, batch_y_los = batch_x_lab.float(),batch_x_lab_length.float() , batch_x_demo.float().to(device), batch_y_outcome.float(), batch_y_los.float()
            batch_y_outcome = batch_y_outcome.unsqueeze(-1)
            batch_y_los = batch_y_los.unsqueeze(-1)
            outcome, los = model(batch_x_lab, batch_x_lab_length, batch_x_demo)
            loss = loss_fn(outcome, los, batch_x_lab_length, batch_y_outcome, batch_y_los)
            val_loss.append(loss.item())
            outcome = torch.squeeze(outcome)
            los = torch.squeeze(los)
            batch_y_outcome = torch.squeeze(batch_y_outcome)
            batch_y_los = torch.squeeze(batch_y_los)
            for i in range(len(batch_y_outcome)):
                y_outcome_pred.extend(outcome[i][:batch_x_lab_length[i].long()].tolist())
                y_outcome_true.extend([batch_y_outcome[i].item()] * batch_x_lab_length[i].long())
                y_los_pred.extend(los[i][:batch_x_lab_length[i].long()].tolist())
                y_los_true.extend(batch_y_los[i][:batch_x_lab_length[i].long()].tolist())
    y_outcome_true = np.array(y_outcome_true)
    y_outcome_pred = np.array(y_outcome_pred)
    y_outcome_pred = np.stack([1 - y_outcome_pred, y_outcome_pred], axis=1)
    outcome_evaluation_scores = metrics.print_metrics_binary(y_outcome_true, y_outcome_pred)
    los_evaluation_scores = metrics.print_metrics_regression(y_los_true, y_los_pred)
    return np.array(val_loss).mean(), outcome_evaluation_scores, los_evaluation_scores

def test_epoch(model, device, dataloader, loss_fn):
    test_loss = []
    y_outcome_pred = []
    y_outcome_true = []
    y_los_pred = []
    y_los_true = []
    evaluation_scores = {}
    model.eval()
    with torch.no_grad():
        for step, data in enumerate(dataloader):   
            batch_x_lab, batch_x_lab_length, batch_x_demo, batch_y_outcome, batch_y_los = data
            batch_x_lab, batch_x_lab_length, batch_x_demo, batch_y_outcome, batch_y_los = batch_x_lab.float(),batch_x_lab_length.float() , batch_x_demo.float().to(device), batch_y_outcome.float(), batch_y_los.float()
            batch_y_outcome = batch_y_outcome.unsqueeze(-1)
            batch_y_los = batch_y_los.unsqueeze(-1)
            outcome, los = model(batch_x_lab, batch_x_lab_length, batch_x_demo)
            loss = loss_fn(outcome, los, batch_x_lab_length, batch_y_outcome, batch_y_los)
            test_loss.append(loss.item())
            outcome = torch.squeeze(outcome)
            los = torch.squeeze(los)
            batch_y_outcome = torch.squeeze(batch_y_outcome)
            batch_y_los = torch.squeeze(batch_y_los)
            for i in range(len(batch_y_outcome)):
                y_outcome_pred.extend(outcome[i][:batch_x_lab_length[i].long()].tolist())
                y_outcome_true.extend([batch_y_outcome[i].item()] * batch_x_lab_length[i].long())
                y_los_pred.extend(los[i][:batch_x_lab_length[i].long()].tolist())
                y_los_true.extend(batch_y_los[i][:batch_x_lab_length[i].long()].tolist())
    y_outcome_true = np.array(y_outcome_true)
    y_outcome_pred = np.array(y_outcome_pred)
    y_outcome_pred = np.stack([1 - y_outcome_pred, y_outcome_pred], axis=1)
    outcome_evaluation_scores = metrics.print_metrics_binary(y_outcome_true, y_outcome_pred)
    los_evaluation_scores = metrics.print_metrics_regression(y_los_true, y_los_pred)
    return np.array(test_loss).mean(), outcome_evaluation_scores, los_evaluation_scores

# train_epoch(model, device, train_loader, get_bce_loss, optimizer)

In [ ]:
class Transformer(nn.Module):
    def __init__(self, input_lab_dim, input_demo_dim, hidden_dim, output_dim, act_layer=nn.GELU, drop=0.):
        super(Transformer, self).__init__()

        # hyperparameters
        self.input_lab_dim = input_lab_dim
        self.input_demo_dim = input_demo_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.demo_proj = nn.Linear(input_demo_dim, hidden_dim)
        self.lab_proj = nn.Linear(input_lab_dim, hidden_dim)

        self.encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=4, dim_feedforward=512, activation='gelu')
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=1)

        self.gru = nn.GRU(input_size = hidden_dim, hidden_size = hidden_dim, num_layers = 1, batch_first = True)
        
        self.act = act_layer()
        # self.fc = nn.Linear(2*hidden_dim, output_dim)
        self.drop = nn.Dropout(drop)

        self.prediction_head_los = nn.Sequential(
            nn.Dropout(drop),
            nn.Linear(2 * hidden_dim, output_dim),
            nn.Dropout(drop),
        )

        self.prediction_head_outcome = nn.Sequential(
            nn.Dropout(drop),
            nn.Linear(2 * hidden_dim, output_dim),
            nn.Dropout(drop),
            nn.Sigmoid(),
        )

    def forward(self, x_lab, x_lab_length, x_demo):
        batch_size, max_length, input_dim = x_lab.shape

        x_lab = self.lab_proj(x_lab)
        x_lab = self.act(x_lab)
        # print('1', x_lab.shape)
        # x_lab, h_n = self.gru(x_lab)
        x_lab = self.transformer_encoder(x_lab)
        # print('2', x_lab.shape)

        x_demo = self.demo_proj(x_demo)
        x_demo = self.act(x_demo) # (batch_size, hidden_dim)

        x_demo = torch.reshape(x_demo.repeat(1, max_length), (batch_size, max_length, self.hidden_dim))

        # print(x_lab.shape, x_demo.shape)
        x = torch.cat((x_lab, x_demo), 2) # (batch_size, 2*hidden_dim)

        outcome = self.prediction_head_outcome(x)
        los =  self.prediction_head_los(x)
        return outcome, los


In [ ]:
model = Transformer(input_lab_dim=25, input_demo_dim=2, hidden_dim=32, output_dim=1, act_layer=nn.GELU, drop=0.).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = predict_all_visits_outcome_bce_and_los_mse_loss
train_loader = DataLoader(train_dataset, batch_size=batch_size)
# train_loss = train_epoch(model, device, train_loader, criterion, optimizer)
train_loss, outcome_evaluation_scores, los_evaluation_scores = test_epoch(model, device, train_loader, criterion)
print('train_loss', train_loss, outcome_evaluation_scores, los_evaluation_scores)

In [ ]:
all_history={}
test_performance={'test_loss': [],'test_mad':[], 'test_mse':[], 'test_mape':[], 'test_accuracy':[], 'test_auroc':[], 'test_auprc':[]}
dataset = train_dataset
num_folds = 5
kfold_test = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=RANDOM_SEED)

for fold_test, (train_and_val_idx, test_idx) in enumerate(kfold_test.split(np.arange(len(dataset)), dataset.y_outcome)):
    print('====== Test Fold {} ======'.format(fold_test + 1))
    kfold_val = StratifiedKFold(n_splits=num_folds-1, shuffle=True, random_state=RANDOM_SEED)
    test_sampler = SubsetRandomSampler(test_idx)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
    sub_dataset = Dataset(dataset.x_lab[train_and_val_idx], dataset.x_lab_length[train_and_val_idx], dataset.x_demo[train_and_val_idx], dataset.y_outcome[train_and_val_idx], dataset.y_los[train_and_val_idx])
    all_history['test_fold_{}'.format(fold_test+1)] = {}
    for fold_val, (train_idx, val_idx) in enumerate(kfold_val.split(np.arange(len(sub_dataset)), sub_dataset.y_outcome)):
        print('Val Fold {}'.format(fold_val + 1))
        train_sampler = SubsetRandomSampler(train_idx)
        val_sampler = SubsetRandomSampler(val_idx)
        train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
        val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)
        model = Transformer(input_lab_dim=25, input_demo_dim=2, hidden_dim=32, output_dim=1, act_layer=nn.GELU, drop=0.).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
        criterion = predict_all_visits_outcome_bce_and_los_mse_loss # get_bce_loss
        history = {'train_loss': [], 'val_loss': [],'val_mad':[], 'val_mse':[], 'val_mape':[]}
        best_val_performance = 1e8
        for epoch in range(num_epochs):
            train_loss=train_epoch(model, device, train_loader, criterion, optimizer)
            val_loss, val_outcome_evaluation_scores, val_los_evaluation_scores=val_epoch(model, device, val_loader, criterion)
            # save performance history on validation set
            print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Val Loss:{:.3f}".format(epoch + 1, num_epochs, train_loss, val_loss))
            history['train_loss'].append(train_loss)
            history['val_loss'].append(val_loss)
            history['val_mad'].append(val_los_evaluation_scores['mad'])
            history['val_mse'].append(val_los_evaluation_scores['mse'])
            history['val_mape'].append(val_los_evaluation_scores['mape'])
            
            # if mad is lower, than set the best mad, save the model, and test it on the test set
            if val_los_evaluation_scores['mad'] < best_val_performance:
                best_val_performance = val_los_evaluation_scores['mad']
                torch.save(model.state_dict(), './checkpoints/transformer_los_best_mad.pth')
        model = Transformer(input_lab_dim=25, input_demo_dim=2, hidden_dim=32, output_dim=1, act_layer=nn.GELU, drop=0.).to(device)
        model.load_state_dict(torch.load('./checkpoints/transformer_los_best_mad.pth'))
        test_loss, test_outcome_evaluation_scores, test_los_evaluation_scores = test_epoch(model, device, test_loader, criterion)
        test_performance['test_loss'].append(test_loss)
        test_performance['test_mad'].append(test_los_evaluation_scores['mad'])
        test_performance['test_mse'].append(test_los_evaluation_scores['mse'])
        test_performance['test_mape'].append(test_los_evaluation_scores['mape'])
        test_performance['test_accuracy'].append(test_outcome_evaluation_scores['acc'])
        test_performance['test_auroc'].append(test_outcome_evaluation_scores['auroc'])
        test_performance['test_auprc'].append(test_outcome_evaluation_scores['auprc'])
        print(f"Performance on test set {fold_test+1}: MAD = {test_los_evaluation_scores['mape']}, MSE = {test_los_evaluation_scores['mse']}, MAPE = {test_los_evaluation_scores['mape']}, Accuracy = {test_outcome_evaluation_scores['acc']}, AUROC = {test_outcome_evaluation_scores['auroc']}, AUPRC = {test_outcome_evaluation_scores['auprc']}")
        all_history['test_fold_{}'.format(fold_test+1)]['fold{}'.format(fold_val+1)] = history


In [ ]:
# Calculate average performance on 10-fold test set
# print(test_performance)
test_mad_list = np.array(test_performance['test_mad'])
test_mse_list = np.array(test_performance['test_mse'])
test_mape_list = np.array(test_performance['test_mape'])
test_accuracy_list = np.array(test_performance['test_accuracy'])
test_auroc_list = np.array(test_performance['test_auroc'])
test_auprc_list = np.array(test_performance['test_auprc'])

print('MAD: mean={:.3f}, std={:.3f}'.format(test_mad_list.mean(), test_mad_list.std()))
print('MSE: mean={:.3f}, std={:.3f}'.format(test_mse_list.mean(), test_mse_list.std()))
print('MAPE: mean={:.3f}, std={:.3f}'.format(test_mape_list.mean(), test_mape_list.std()))
print('accuracy: mean={:.3f}, std={:.3f}'.format(test_accuracy_list.mean(), test_accuracy_list.std()))
print('auroc: mean={:.3f}, std={:.3f}'.format(test_auroc_list.mean(), test_auroc_list.std()))
print('auprc: mean={:.3f}, std={:.3f}'.format(test_auprc_list.mean(), test_auprc_list.std()))


"""
MAD: mean=4.110, std=0.454
MSE: mean=37.216, std=8.237
MAPE: mean=352.299, std=106.865
accuracy: mean=0.878, std=0.059
auroc: mean=0.947, std=0.043
auprc: mean=0.912, std=0.074
"""